In [1]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torch
from torchvision import transforms
import import_ipynb
from config import batch_size, rootDir, device, lr, epochs, base_output
from data import AFAD
from model import MTL
from glob import glob
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

importing Jupyter notebook from config.ipynb
importing Jupyter notebook from data.ipynb
importing Jupyter notebook from model.ipynb


fatal: destination path 'tarball-lite' already exists and is not an empty directory.


In [2]:
def save_loss_plot(out_dir, train_loss, val_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    train_ax.plot(train_loss, color='tab:blue')
    train_ax.set_xlabel('iterations')
    train_ax.set_ylabel('train loss')
    valid_ax.plot(val_loss, color='tab:red')
    valid_ax.set_xlabel('iterations')
    valid_ax.set_ylabel('validation loss')
    figure_1.savefig(f"{out_dir}/train_loss.png")
    figure_2.savefig(f"{out_dir}/valid_loss.png")
    print('SAVING PLOTS COMPLETE...')
    
    plt.close('all')

In [3]:
images_path = []
ages = []
genders = []

for age in os.listdir(rootDir):
    files = os.path.join(rootDir, age)
    if os.path.isdir(files):
        for gender in os.listdir(files):
            files = os.path.join(files, gender)
            if os.path.isdir(files):
                for img in os.listdir(files):
                    image_path = os.path.join(files, img)
                    images_path.append(image_path)
                    ages.append(int(age))
                    genders.append(int(gender[2])-1)

In [4]:
train_paths, val_paths = train_test_split(
    images_path, test_size=0.1, random_state=10
)
train_ages, val_ages = train_test_split(
    ages, test_size=0.1, random_state=10
)
train_genders, val_genders = train_test_split(
    genders, test_size=0.1, random_state=10
)
train_dataloader = DataLoader(
    AFAD(train_paths, train_ages, train_genders), shuffle=True, batch_size=batch_size
)
val_dataloader = DataLoader(
    AFAD(val_paths, val_ages, val_genders), shuffle=False, batch_size=batch_size
)

In [5]:
model = MTL().to(device=device)
opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.09)
gender_loss = nn.BCELoss() # output必須要Sigmoid
age_loss = nn.L1Loss()
Sig = nn.Sigmoid()

C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
C:\Users\x9172\anaconda3\envs\PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
H = {"train_loss": [], "valid_loss": []}

In [7]:
for epoch in range(epochs):
    model.train()
    training_loss = 0
    for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs = data["image"].to(device=device)
        age_label = data["age"].to(device=device)
        gender_label = data["gender"].to(device=device)
        
        opt.zero_grad()
        
        age_output, gender_output = model(inputs)
        loss_1 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
        loss_2 = age_loss(age_output, age_label.unsqueeze(1).float())
        loss = loss_1 + loss_2
        loss.backward()
        opt.step()
        training_loss += loss.item()
    avgTrainLoss = training_loss/ len(train_dataloader)
    print(  
            'epoch: {} epoch_loss {}'.format(
                epoch+1,
                avgTrainLoss,
            )
        )
    H["train_loss"].append(avgTrainLoss)
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        for i, data in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
            inputs = data["image"].to(device=device)
            age_label = data["age"].to(device=device)
            gender_label = data["gender"].to(device=device)

            opt.zero_grad()

            age_output, gender_output = model(inputs)
            loss_1 = gender_loss(Sig(gender_output), gender_label.unsqueeze(1).float())
            loss_2 = age_loss(age_output, age_label.unsqueeze(1).float())
            loss = loss_1 + loss_2
            valid_loss += loss.item()
        avgValidLoss = valid_loss / len(val_dataloader)
        print(  
            'epoch: {} epoch_loss {}'.format(
                epoch+1,
                avgValidLoss,
            )
        )
        H["valid_loss"].append(avgValidLoss)
        
        save_loss_plot(
                base_output, H["train_loss"],  H["valid_loss"]
        )
        torch.save(model.state_dict(), f'{base_output}/model.pth')

100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [00:54<00:00,  4.49it/s]


epoch: 1 epoch_loss 5.41042220835783


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:04<00:00,  6.45it/s]


epoch: 1 epoch_loss 4.498956177915845
SAVING PLOTS COMPLETE...


100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [00:52<00:00,  4.69it/s]


...

epoch: 24 epoch_loss 3.896452682358878
SAVING PLOTS COMPLETE...


100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [00:52<00:00,  4.67it/s]


epoch: 25 epoch_loss 2.6604844750190266


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:04<00:00,  6.45it/s]


epoch: 25 epoch_loss 4.49388302224023
SAVING PLOTS COMPLETE...
